In [11]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [1]:
output = open('clinica_output_1.txt', 'r').readlines()

In [16]:
bad_file_list = [output.split()[0] for output in output]

In [9]:
df = pd.read_csv('/gpfs/home/lc3424/capstone/2021_dementia/lc3424_workspace/' + 'file_migration_table_t1_v2.tsv', sep='\t')
df

,sess_info,file_type,de-identified acc,random_pat_id,scan_date_time,label,orig_path,dest_path
0,-20150220-AX_3D_MPR-22,json,2.595211e+11,5c2f5ed8aeee4c4ba21955ce3b148740,2017-06-03 12:30:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20150220-AX_3D_MPR-22.json
1,-20150220-AX_3D_MPR-22,niigz,2.595211e+11,5c2f5ed8aeee4c4ba21955ce3b148740,2017-06-03 12:30:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20150220-AX_3D_MPR-22.nii.gz
2,-20150220-SAG_MPR_RECON_1MM-23,json,2.595211e+11,5c2f5ed8aeee4c4ba21955ce3b148740,2017-06-03 12:30:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20150220-SAG_MPR_RECON_1MM-23.json
3,-20150220-SAG_MPR_RECON_1MM-23,niigz,2.595211e+11,5c2f5ed8aeee4c4ba21955ce3b148740,2017-06-03 12:30:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20150220-SAG_MPR_RECON_1MM-23.nii.gz
4,1003422834-20080613-AX_3D_MPR_ISO-13,json,7.862150e+11,644e107e27d540c2a65f159f45fbfe81,2015-08-27 09:56:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20080613-AX_3D_MPR_ISO-13.json
...,...,...,...,...,...,...,...,...
21051,9999070215-20190719-AX_MPR_MPR_sag_recon-18001,niigz,3.160603e+11,16a191ad3fae43ac9be96c18cd86194a,2020-09-16 09:34:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20190719-AX_MPR_MPR_sag_recon-18001.nii.gz
21052,9999070215-20191225-AX_3D_MPR-13,json,2.585815e+11,16a191ad3fae43ac9be96c18cd86194a,2021-02-22 08:20:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20191225-AX_3D_MPR-13.json
21053,9999070215-20191225-AX_3D_MPR-13,niigz,2.585815e+11,16a191ad3fae43ac9be96c18cd86194a,2021-02-22 08:20:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20191225-AX_3D_MPR-13.nii.gz
21054,9999070215-20191225-SAG_MPR_RECON-101,json,2.585815e+11,16a191ad3fae43ac9be96c18cd86194a,2021-02-22 08:20:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20191225-SAG_MPR_RECON-101.json


In [23]:
output_dir = '/gpfs/data/razavianlab/data/mri/nyu/barlow_bids_t1/'
id_summary = set()
historical_sess = defaultdict(int)
historical_sess_info = defaultdict(int)

for idx, row in df.iterrows():
    # get patient id
    id = row.random_pat_id
    # create patient folders
    pat_folder_name = output_dir + 'sub-{}/'.format(id)

    # create session folder
    sess = int(row['de-identified acc'])
    if row.sess_info not in historical_sess_info:
        historical_sess[sess] += 1
        historical_sess_info[row.sess_info] = historical_sess[sess]
    if historical_sess[sess] > 1:
        sess = str(sess) + '_{}'.format(historical_sess_info[row.sess_info])
    
    ses_name = 'ses-{}/'.format(sess)
    sess_folder_name = pat_folder_name + ses_name
    if not os.path.exists(sess_folder_name):
        os.mkdir(sess_folder_name)
    if not os.path.exists(sess_folder_name + 'anat'):
        os.mkdir(sess_folder_name + 'anat')

    cur_file_name = row.dest_path.replace('.nii.gz', '_T1w.nii.gz').replace('.json', '_T1w.json')
    file_name = 'sub-{}_ses-{}_'.format(id, sess) + cur_file_name
    df.loc[idx, 'file_name'] = file_name

In [24]:
df_img = df[df.file_type == 'niigz']

In [30]:
df_img[~df_img.file_name.isin(bad_file_list)]['de-identified acc'].unique().shape

(3739,)

In [32]:
df_img = df_img[~df_img.file_name.isin(bad_file_list)]

In [33]:
df_img

,sess_info,file_type,de-identified acc,random_pat_id,scan_date_time,label,orig_path,dest_path,file_name
1,-20150220-AX_3D_MPR-22,niigz,2.595211e+11,5c2f5ed8aeee4c4ba21955ce3b148740,2017-06-03 12:30:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20150220-AX_3D_MPR-22.nii.gz,sub-5c2f5ed8aeee4c4ba21955ce3b148740_ses-25952...
3,-20150220-SAG_MPR_RECON_1MM-23,niigz,2.595211e+11,5c2f5ed8aeee4c4ba21955ce3b148740,2017-06-03 12:30:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20150220-SAG_MPR_RECON_1MM-23.nii.gz,sub-5c2f5ed8aeee4c4ba21955ce3b148740_ses-25952...
5,1003422834-20080613-AX_3D_MPR_ISO-13,niigz,7.862150e+11,644e107e27d540c2a65f159f45fbfe81,2015-08-27 09:56:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20080613-AX_3D_MPR_ISO-13.nii.gz,sub-644e107e27d540c2a65f159f45fbfe81_ses-78621...
7,1003422834-20080613-AX_MPR_RECON-101,niigz,7.862150e+11,644e107e27d540c2a65f159f45fbfe81,2015-08-27 09:56:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20080613-AX_MPR_RECON-101.nii.gz,sub-644e107e27d540c2a65f159f45fbfe81_ses-78621...
9,1003422834-20080613-SAG_MPR_RECON-103,niigz,7.862150e+11,644e107e27d540c2a65f159f45fbfe81,2015-08-27 09:56:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20080613-SAG_MPR_RECON-103.nii.gz,sub-644e107e27d540c2a65f159f45fbfe81_ses-78621...
...,...,...,...,...,...,...,...,...,...
21043,9999070215-20190313-AX_3D_MPR-12,niigz,1.185208e+11,16a191ad3fae43ac9be96c18cd86194a,2020-05-11 10:33:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20190313-AX_3D_MPR-12.nii.gz,sub-16a191ad3fae43ac9be96c18cd86194a_ses-11852...
21045,9999070215-20190313-SAG_MPR_RECON-101,niigz,1.185208e+11,16a191ad3fae43ac9be96c18cd86194a,2020-05-11 10:33:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20190313-SAG_MPR_RECON-101.nii.gz,sub-16a191ad3fae43ac9be96c18cd86194a_ses-11852...
21047,9999070215-20190719-AX_MPR-16001,niigz,3.160603e+11,16a191ad3fae43ac9be96c18cd86194a,2020-09-16 09:34:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20190719-AX_MPR-16001.nii.gz,sub-16a191ad3fae43ac9be96c18cd86194a_ses-31606...
21053,9999070215-20191225-AX_3D_MPR-13,niigz,2.585815e+11,16a191ad3fae43ac9be96c18cd86194a,2021-02-22 08:20:00,0,/gpfs/data/razavianlab/data/mri/nyu/barlow_nii...,20191225-AX_3D_MPR-13.nii.gz,sub-16a191ad3fae43ac9be96c18cd86194a_ses-25858...
